https://github.com/zhangxu0307/time_series_forecasting_pytorch

In [136]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import pandas as pd
import matplotlib.pyplot as plt

# Load Data

In [137]:
obs_cols = ["frame_number", "pedestrian_ID", "pos_x", "pos_z", "pos_y", "v_x", "v_z", "v_y"]
url = "https://raw.githubusercontent.com/AzucenaMV/SocialLSTM/master/eth/eth/obsmat.txt"
df = pd.read_csv(url, delimiter='\s+', header=None, names = obs_cols)

In [138]:
df.head(2)

,frame_number,pedestrian_ID,pos_x,pos_z,pos_y,v_x,v_z,v_y
0,780.0,1.0,8.456844,0.0,3.588066,1.671714,0.0,0.176292
1,786.0,1.0,9.125530,0.0,3.658583,1.662877,0.0,0.326723


In [139]:
threshold_frames = 15
ids = df.pedestrian_ID.unique()
ids_index = [len(df[df.pedestrian_ID == ped]) >= threshold_frames for ped in ids]
data_clean = df[df.pedestrian_ID.isin(ids[ids_index])]

In [157]:
tw = 10
data_gp_id = data_clean.sort_values(['frame_number']).groupby(['pedestrian_ID'])
train_xy = []
train_xy_target = []
for group_name, data_group in data_gp_id:
    L = len(data_group)
    for i in range(L-tw):
        train_xy.extend(list(zip(data_group.pedestrian_ID[i:i+tw],
                                 data_group.pos_x[i:i+tw],
                                 data_group.pos_y[i:i+tw])))
        train_xy_target.extend(list(zip(data_group.pedestrian_ID[i+tw:i+tw+1],
                                 data_group.pos_x[i+tw:i+tw+1],
                                 data_group.pos_y[i+tw:i+tw+1])))

In [159]:
data_features = pd.DataFrame(train_xy, columns = ['ID','x','y'])
data_targets = pd.DataFrame(train_xy_target, columns = ['ID','x','y'])

In [181]:
num_steps_obs = 10
num_steps_pred = 1
obs = len(data_features)//num_steps_obs
data_features_reshape = data_features.drop(columns=['ID']) \
            .values.reshape(obs, num_steps_obs, 2)
data_targets_reshape = data_targets.drop(columns=['ID']) \
            .values.reshape(obs, 2)

# train 
X_train = data_features_reshape[1:4500]
print(X_train.shape)
Y_train = data_targets_reshape[1:4500]
print(Y_train.shape)
# test
X_test = data_features_reshape[4500:]
print(X_test.shape)
Y_test = data_targets_reshape[4500:]
print(Y_test.shape)

(4499, 10, 2)
(4499, 2)
(851, 10, 2)
(851, 2)


In [182]:
# cast into float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
Y_train = Y_train.astype('float32')
Y_test = Y_test.astype('float32')

# RNN

In [183]:
from torch.utils.data import Dataset
class Time_Series_Data(Dataset):

    def __init__(self, train_x, train_y):
        self.X = train_x
        self.y = train_y

    def __getitem__(self, item):
        x_t = self.X[item]
        y_t = self.y[item]
        return x_t, y_t

    def __len__(self):

        return len(self.X)



In [184]:
# RNNs模型基类，主要是用于指定参数和cell类型
class BaseModel(nn.Module):

    def __init__(self, inputDim, hiddenNum, outputDim, layerNum, cell, use_cuda=False):

        super(BaseModel, self).__init__()
        self.hiddenNum = hiddenNum
        self.inputDim = inputDim
        self.outputDim = outputDim
        self.layerNum = layerNum
        self.use_cuda = use_cuda
        if cell == "RNN":
            self.cell = nn.RNN(input_size=self.inputDim, hidden_size=self.hiddenNum,
                        num_layers=self.layerNum, dropout=0.0,
                         nonlinearity="tanh", batch_first=True,)
        if cell == "LSTM":
            self.cell = nn.LSTM(input_size=self.inputDim, hidden_size=self.hiddenNum,
                               num_layers=self.layerNum, dropout=0.0,
                               batch_first=True, )
        if cell == "GRU":
            self.cell = nn.GRU(input_size=self.inputDim, hidden_size=self.hiddenNum,
                                num_layers=self.layerNum, dropout=0.0,
                                 batch_first=True, )
        print(self.cell)
        #fc - fully connected
        self.fc = nn.Linear(self.hiddenNum, self.outputDim) #arg: size of input sample, size of output sample
        #the above are setups, these parameters are different from the ones passed in below

# 标准RNN模型
class RNNModel(BaseModel):

    def __init__(self, inputDim, hiddenNum, outputDim, layerNum, cell, use_cuda):

        super(RNNModel, self).__init__(inputDim, hiddenNum, outputDim, layerNum, cell, use_cuda)
        
    #most confused here about the dimensions
    def forward(self, x):
        
        batchSize = x.size(0) #x.dim1 is batchsize 32, x's dim2 should be time series sequence length 24. x(32,24)

        h0 = Variable(torch.zeros(self.layerNum * 1, batchSize , self.hiddenNum)) #variable includes backward calc arg
        #so it is a 3D - for each layer, each batchsize, each hidden neuron
        if self.use_cuda:
            h0 = h0.cuda()
        rnnOutput, hn = self.cell(x, h0) #?pass in x and a 3D zero tensor? (1,32,64) as h0 input
        hn = hn.view(batchSize, self.hiddenNum)  #(32,64)
        fcOutput = self.fc(hn) #pass in hidden layer to give linear estimation

        return fcOutput

In [185]:
# LSTM模型
class LSTMModel(BaseModel):

    def __init__(self, inputDim, hiddenNum, outputDim, layerNum, cell, use_cuda):
        super(LSTMModel, self).__init__(inputDim, hiddenNum, outputDim, layerNum, cell, use_cuda)

    def forward(self, x):

        batchSize = x.size(0)
        h0 = Variable(torch.zeros(self.layerNum * 1, batchSize, self.hiddenNum))
        c0 = Variable(torch.zeros(self.layerNum * 1, batchSize, self.hiddenNum))
        if self.use_cuda:
            h0 = h0.cuda()
            c0 = c0.cuda()
        rnnOutput, hn = self.cell(x, (h0, c0))
        hn = hn[0].view(batchSize, self.hiddenNum)
        fcOutput = self.fc(hn)

        return fcOutput

In [186]:
def train(trainX, trainY,  lag, lr, method, hidden_num=64, epoch=20, batchSize=32,
           checkPoint=10, use_cuda=False): #hidden_num here is 64, what kind of dimension is this?

    lossList = []

    # build up data loader
    dataset = Time_Series_Data(trainX, trainY)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, sampler=None,
                                             batch_sampler=None, num_workers=1)
    net = None
    if method == "RNN": #seems these dim means number of neurons not the dimension of data
        net = RNNModel(inputDim=2, hiddenNum=hidden_num, outputDim=2, layerNum=1, cell="RNN", use_cuda=use_cuda)
    if method == "LSTM":
        net = LSTMModel(inputDim=2, hiddenNum=hidden_num, outputDim=2, layerNum=1, cell="LSTM", use_cuda=use_cuda)
    if method == "GRU":
        net = GRUModel(inputDim=2, hiddenNum=hidden_num, outputDim=2, layerNum=1, cell="GRU", use_cuda=use_cuda)
    if method == "ResRNN":
        net = ResRNNModel(inputDim=2, hiddenNum=hidden_num, outputDim=2, resDepth=4, use_cuda=use_cuda)
    # if method == "attention":
    #     net = RNN_Attention(inputDim=1, hiddenNum=hidden_num, outputDim=1, resDepth=4,
    #                         seq_len=lag, merge="concate", use_cuda=use_cuda)
    if method == "MLP":
        net = MLPModel(inputDim=lag, hiddenNum=hidden_num, outputDim=1)
    if use_cuda:
        net = net.cuda()
    net = net.train() #just to let the function know you are training the model.
    #So effectively layers like dropout, batchnorm etc. which behave different on the train and test procedures know what is going on and hence can behave accordingly.
    optimizer = optim.RMSprop(net.parameters(), lr=lr, momentum=0.9)
    criterion = nn.MSELoss()

    t1 = time.time()
    lossSum = 0

    print("data loader num:", len(dataloader))

    for i in range(epoch):

        for batch_idx, (x, y) in enumerate(dataloader):
            #set them into variables so that their backpropagation is on
            x, y = Variable(x), Variable(y)
            if use_cuda:
                x = x.cuda()
                y = y.cuda()

            optimizer.zero_grad()

            pred = net.forward(x)
            loss = criterion(pred, y)

            lossSum += loss.item()
            if batch_idx % checkPoint == 0 and batch_idx != 0:
               print("batch: %d , loss is:%f" % (batch_idx, lossSum / checkPoint))
               lossList.append(lossSum / checkPoint)
               lossSum = 0

            loss.backward()
            optimizer.step()

        print("%d epoch is finished!" % (i+1))

    t2 = time.time()
    print("train time:", t2-t1)
    #p.dump(lossList, output, -1)

    return net


In [187]:
# trainX needs to be 3D array (non tensor), float32
trained_RNN = train(X_train, 
                    Y_train, 
                    10, 
                    1e-3 , 
                    'RNN', 
                    hidden_num=30, 
                    epoch=20, 
                    batchSize=100,
                    checkPoint=10, 
                    use_cuda=False) 

RNN(2, 30, batch_first=True)
data loader num: 45
batch: 10 , loss is:22.211610
batch: 20 , loss is:5.198295
batch: 30 , loss is:2.411288
batch: 40 , loss is:1.567796
1 epoch is finished!
batch: 10 , loss is:1.272195
batch: 20 , loss is:0.501405
batch: 30 , loss is:0.285970
batch: 40 , loss is:0.250425
2 epoch is finished!
batch: 10 , loss is:0.238179
batch: 20 , loss is:0.125735
batch: 30 , loss is:0.098248
batch: 40 , loss is:0.118173
3 epoch is finished!
batch: 10 , loss is:0.125195
batch: 20 , loss is:0.074534
batch: 30 , loss is:0.058537
batch: 40 , loss is:0.054384
4 epoch is finished!
batch: 10 , loss is:0.075220
batch: 20 , loss is:0.047672
batch: 30 , loss is:0.039660
batch: 40 , loss is:0.035501
5 epoch is finished!
batch: 10 , loss is:0.054807
batch: 20 , loss is:0.035633
batch: 30 , loss is:0.032014
batch: 40 , loss is:0.034855
6 epoch is finished!
batch: 10 , loss is:0.044967
batch: 20 , loss is:0.030590
batch: 30 , loss is:0.028059
batch: 40 , loss is:0.024218
7 epoch is f

In [188]:
trained_LSTM = train(X_train, 
                     Y_train, 
                     10, 
                     1e-3 , 
                     'LSTM', 
                     hidden_num=30, 
                     epoch=10, batchSize=100,checkPoint=10, use_cuda=False) 

LSTM(2, 30, batch_first=True)
data loader num: 45
batch: 10 , loss is:24.993407
batch: 20 , loss is:9.579905
batch: 30 , loss is:3.737527
batch: 40 , loss is:2.676753
1 epoch is finished!
batch: 10 , loss is:2.442253
batch: 20 , loss is:1.473218
batch: 30 , loss is:1.281362
batch: 40 , loss is:1.071958
2 epoch is finished!
batch: 10 , loss is:1.022177
batch: 20 , loss is:0.397339
batch: 30 , loss is:0.249996
batch: 40 , loss is:0.179157
3 epoch is finished!
batch: 10 , loss is:0.162362
batch: 20 , loss is:0.083609
batch: 30 , loss is:0.099135
batch: 40 , loss is:0.069578
4 epoch is finished!
batch: 10 , loss is:0.101196
batch: 20 , loss is:0.059782
batch: 30 , loss is:0.059474
batch: 40 , loss is:0.055749
5 epoch is finished!
batch: 10 , loss is:0.058919
batch: 20 , loss is:0.048427
batch: 30 , loss is:0.046805
batch: 40 , loss is:0.048924
6 epoch is finished!
batch: 10 , loss is:0.058464
batch: 20 , loss is:0.029346
batch: 30 , loss is:0.032093
batch: 40 , loss is:0.036970
7 epoch is 

# Prediction

In [457]:
id_test = data_targets.loc[:4500,'ID'].unique()
data_test_clean = data_clean[data_clean.pedestrian_ID.isin(id_test)]\
    .sort_values(['frame_number']).groupby('pedestrian_ID').head(15)
test_obs = len(data_test_clean)//15
X_test = data_test_clean[['pos_x','pos_y']].values.reshape(test_obs,15,2)

In [413]:
data_test_clean = data_clean[data_clean.pedestrian_ID.isin(id_test)]\
    .sort_values(['frame_number']).groupby('pedestrian_ID').head(10)
test_obs = len(data_test_clean)//10
X_pred = data_test_clean[['pos_x','pos_y']].values.reshape(test_obs,10,2)

In [464]:
id_test = data_targets.loc[:4500,'ID'].unique()
data_test_clean = data_clean[data_clean.pedestrian_ID.isin(id_test)]\
    .sort_values(['frame_number']).groupby('pedestrian_ID').head(15)

In [465]:
data_test_clean['seq'] = data_test_clean.groupby(['pedestrian_ID']).cumcount()+1

In [470]:
x_test = data_test_clean.pivot(index='pedestrian_ID', columns='seq', values='pos_x').reset_index()
y_test = data_test_clean.pivot(index='pedestrian_ID', columns='seq', values='pos_y').reset_index()

In [471]:
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

ValueError: could not determine the shape of object type 'DataFrame'

In [403]:
tw = 10
trained_LSTM.eval()
X_pred = torch.FloatTensor(X_pred)
for i in range(fut_pred):
    seq = torch.FloatTensor(X_pred[:,-tw:,:])
    with torch.no_grad():
            #model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
            #                torch.zeros(1, 1, model.hidden_layer_size
        pred = trained_LSTM(seq)
    X_pred = torch.cat([X_pred, pred.reshape(270,1,2)], dim=1)

# Evaluation

In [120]:
#encoding=utf-8
from sklearn.metrics import mean_squared_error,mean_absolute_error
import numpy as np


# 计算RMSE
def calcRMSE(true,pred):
    return np.sqrt(mean_squared_error(true, pred))


# 计算MAE
def calcMAE(true,pred):
    return mean_absolute_error(true, pred)


# 计算MAPE
def calcMAPE(true, pred, epsion = 0.0000000):

    true += epsion
    return np.mean(np.abs((true-pred)/true))*100


# 计算SMAPE
def calcSMAPE(true, pred):
    delim = (np.abs(true)+np.abs(pred))/2.0
    return np.mean(np.abs((true-pred)/delim))*100

In [130]:
print(calcRMSE(Y_train, trainy_pred))
print(calcRMSE(Y_test, testy_pred))

0.3437374
0.3446669


In [129]:
print(calcRMSE(Y_train, trainy_pred_lstm))
print(calcRMSE(Y_test, testy_pred_lstm))

0.26651198
0.3147586
